In [11]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))

from data.data_loader import TimeSeriesDataLoader
import pandas as pd
import numpy as np
from layers.multi_cp import SPCI_and_EnbPI
from models.lstm import LSTMModel

print("Data loader imported successfully!")

Data loader imported successfully!


In [12]:
df = pd.read_csv('./data/snp50.csv', index_col='Date')
df

,MSFT,AAPL,JNJ,XOM,JPM,PG,CVX,MRK,KO,PEP,...,KR,COP,UPS,NOC,KMB,MO,ADM,GIS,PPG,DE
Date,,,,,,,,,,,,,,,,,,,,,
2005-01-03,18.454880,0.949987,34.751911,25.093662,22.368937,31.104504,23.147207,14.135262,11.093916,28.976854,...,6.069591,16.229395,44.609112,31.197931,30.214354,4.115128,13.337941,12.691508,21.149090,24.427019
2005-01-04,18.523895,0.959743,34.641403,24.923332,22.138399,30.715654,22.988049,14.076481,10.939011,28.770435,...,5.906789,16.123270,44.042339,30.529194,30.228394,4.143637,13.192176,12.652571,20.671089,23.844303
2005-01-05,18.482491,0.968149,34.619312,24.793068,22.184502,31.036873,23.138121,14.171439,10.888270,28.803917,...,5.857239,16.242908,43.953110,30.994419,29.708813,4.136169,12.967444,12.912111,20.378616,23.613228
2005-01-06,18.461790,0.968900,34.718746,25.108692,22.311295,31.189054,23.524668,14.270920,10.979074,29.010326,...,5.811234,16.478315,43.905884,30.913008,30.190958,4.106984,13.234690,13.006134,20.667944,23.941425
2005-01-07,18.406567,1.039446,34.591671,24.943365,22.132635,31.515942,23.260893,14.076481,10.992427,29.261377,...,5.885553,16.374115,43.821911,30.849022,30.022442,4.167390,13.265059,13.303864,20.768570,23.609882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-23,432.871429,254.367020,141.850220,103.433617,233.448151,164.928162,138.162003,96.588165,61.017918,146.899445,...,60.064949,94.021751,119.708084,461.087036,127.804871,49.940445,48.925728,61.532398,117.926628,426.387177
2024-12-24,436.929138,257.286652,142.416565,103.530914,237.286880,165.742447,139.002731,96.665916,61.467873,148.375351,...,60.922173,94.733894,120.193581,464.926147,128.524399,49.837914,49.139256,61.716370,118.741272,426.840820
2024-12-26,435.715790,258.103729,142.152908,103.618484,238.099686,166.939377,139.138031,97.074165,61.203766,148.016083,...,61.838516,94.519279,120.288773,466.051239,129.448135,49.980442,49.119843,61.735729,118.986649,427.826935


In [13]:
df = df.pct_change()

In [14]:
data_loader = TimeSeriesDataLoader()
train_loader, val_loader, test_loader, dates, scaler = data_loader.create_dataloaders(
    frequency='daily',
    lookback=63,
    forecast_horizon=1,
    train_end_date='2015-12-31',
    val_end_date='2019-12-31',
    test_end_date='2024-12-31',
    batch_size=32,
    use_scaler=True
)

Loaded data: (5031, 50) (2005-01-04 00:00:00 ~ 2024-12-30 00:00:00)
Preprocessed (fit up to 2015-12-31): (5031, 50)
Created sequences - X: (4968, 63, 50), y: (4968, 1, 50)

Data split by date:
Train: 2705 samples (2005-04-06 00:00:00 ~ 2015-12-31 00:00:00)
Val:   1006 samples (2016-01-04 00:00:00 ~ 2019-12-31 00:00:00)
Test:  1257 samples (2020-01-02 00:00:00 ~ 2024-12-30 00:00:00)

DataLoaders created successfully!
Batch size: 32
Train batches: 85
Val batches: 32
Test batches: 40


In [15]:
for X_batch, y_batch in train_loader:
    print("Input batch shape:", X_batch.shape)
    print("Target batch shape:", y_batch.shape)
    
    print("Input batch sample:", X_batch[0])
    print("Target batch sample:", y_batch[0])
    break

Input batch shape: torch.Size([32, 63, 50])
Target batch shape: torch.Size([32, 1, 50])
Input batch sample: tensor([[-0.1224,  1.1336, -0.1821,  ..., -0.5703,  0.1525, -0.1346],
        [-0.1413, -1.0760,  0.4554,  ..., -0.0120,  0.4268,  0.3426],
        [-0.2165, -2.8941,  1.2057,  ..., -0.1936, -1.5884,  0.0982],
        ...,
        [ 0.8632, -0.2616,  0.0137,  ..., -0.3474,  0.6183, -0.2795],
        [ 0.1141, -1.0307,  0.6513,  ..., -0.0770, -0.3288,  0.3857],
        [ 0.2156,  0.5385,  1.0264,  ...,  0.1467,  0.8667,  0.3309]])
Target batch sample: tensor([[ 0.2146, -0.6039,  2.3619,  0.1620,  0.1344,  0.2148,  0.0391, -0.4013,
          2.2251,  1.7467, -0.1260,  0.7406,  0.6936, -0.0598,  1.4721,  0.4192,
          0.1553,  1.3002,  0.0817,  0.2880,  0.5306,  0.5615,  0.0521,  0.4128,
         -0.2519,  0.1275, -0.4455, -0.4414,  0.1521,  0.5335, -0.0804, -0.2480,
          0.0295, -0.2706, -0.2784,  0.0506,  0.1203, -0.1763, -0.7946, -0.2175,
         -0.1615, -0.1483,  0.53

In [16]:
X_train, Y_train = train_loader.dataset.X, train_loader.dataset.y
X_valid, Y_valid = val_loader.dataset.X, val_loader.dataset.y
X_predict, Y_predict = test_loader.dataset.X, test_loader.dataset.y

conformal_predictor = SPCI_and_EnbPI(
    X_train, X_valid, X_predict,
    Y_train, Y_valid, Y_predict,
    model_cls=LSTMModel, 
    scaler=scaler,
    loader = data_loader
)

In [17]:
X_valid.size()

torch.Size([1006, 63, 50])

In [18]:
Y_predict.size()

torch.Size([1257, 1, 50])

In [19]:
results = conformal_predictor.fit_bootstrap_models_online_multistep(B=20, EPOCHS=50) # B is the number of bootstrap models
conformal_predictor.compute_Widths_Ensemble_online(alpha=0.05, smallT=False, use_SPCI=True) # alpha is your significance level
# conformal_predictor.use_local_ellipsoid = True
conformal_predictor.get_results()
# 4. Call the new method to get results in the original scale
# original_scale_results = conformal_predictor.()

# # You can now access the results
# if original_scale_results:
#     original_predictions = original_scale_results['predictions']
#     original_lower_bounds = original_scale_results['lower_bounds']
#     original_upper_bounds = original_scale_results['upper_bounds']
    
#     print("\nSample of inverse-transformed results:")
#     print("Predictions:", original_predictions[0])
#     print("Lower Bounds:", original_lower_bounds[0])
#     print("Upper Bounds:", original_upper_bounds[0])

Model Num 0 (LSTMModel), Epoch 1, Train Loss: 1.1139, Valid Loss: 0.6939
Model Num 0 (LSTMModel), Epoch 2, Train Loss: 1.0249, Valid Loss: 0.6930
Model Num 0 (LSTMModel), Epoch 3, Train Loss: 1.0154, Valid Loss: 0.6923
Model Num 0 (LSTMModel), Epoch 4, Train Loss: 0.9963, Valid Loss: 0.6929
Model Num 0 (LSTMModel), Epoch 5, Train Loss: 1.0006, Valid Loss: 0.6925
Model Num 0 (LSTMModel), Epoch 6, Train Loss: 1.0037, Valid Loss: 0.6923
Model Num 0 (LSTMModel), Epoch 7, Train Loss: 1.0083, Valid Loss: 0.6928
Model Num 0 (LSTMModel), Epoch 8, Train Loss: 0.9872, Valid Loss: 0.6935
Model Num 0 (LSTMModel), Epoch 9, Train Loss: 0.9858, Valid Loss: 0.6954
Model Num 0 (LSTMModel), Epoch 10, Train Loss: 1.0112, Valid Loss: 0.6997
Model Num 0 (LSTMModel), Epoch 11, Train Loss: 0.9998, Valid Loss: 0.7009
Model Num 0 (LSTMModel), Epoch 12, Train Loss: 0.9693, Valid Loss: 0.6986
Model Num 0 (LSTMModel), Epoch 13, Train Loss: 0.9777, Valid Loss: 0.6999
Early stopping for model 0 (LSTMModel) at epoch

(np.float64(0.8090692124105012), np.float64(1.4185378676762372e-34))

In [20]:
df[df.index >'2015-12-31']

,MSFT,AAPL,JNJ,XOM,JPM,PG,CVX,MRK,KO,PEP,...,KR,COP,UPS,NOC,KMB,MO,ADM,GIS,PPG,DE
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,-0.012257,0.000855,-0.021807,-0.006286,-0.030035,-0.013096,-0.012339,-0.006437,-0.013036,-0.011509,...,-0.016017,0.004069,-0.014445,-0.006886,-0.006441,-0.014087,-0.022901,-0.003468,-0.008703,-0.002491
2016-01-05,0.004562,-0.025059,0.004180,0.008520,0.001729,0.003190,0.008554,0.012766,0.003538,0.006885,...,0.022595,0.011092,0.009911,0.026026,0.020556,0.020213,0.011161,-0.004525,0.003573,0.005783
2016-01-06,-0.018165,-0.019570,-0.005055,-0.008321,-0.014436,-0.009667,-0.039505,-0.013735,-0.005405,0.000302,...,-0.000951,-0.043249,-0.014199,-0.009980,-0.000465,0.010589,-0.024283,-0.008217,-0.018716,-0.011239
2016-01-07,-0.034783,-0.042205,-0.011654,-0.016006,-0.040439,-0.008734,-0.035436,-0.008775,-0.016541,-0.019200,...,-0.023305,-0.028445,-0.019276,-0.012390,-0.016742,-0.017408,-0.023755,-0.020963,-0.012647,-0.008855
2016-01-08,0.003067,0.005288,-0.010683,-0.020202,-0.022399,-0.015677,-0.010721,-0.016936,-0.002643,-0.003689,...,-0.006574,-0.017477,-0.013067,-0.010845,-0.012849,0.001720,-0.003766,0.003266,-0.007979,-0.016402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-23,-0.003092,0.003065,0.005538,0.004062,0.003325,0.000298,0.000840,0.013463,-0.002718,-0.009817,...,-0.014390,0.013246,0.000557,-0.004984,0.000914,-0.004829,-0.001585,-0.000943,-0.000499,-0.000254
2024-12-24,0.009374,0.011478,0.003993,0.000941,0.016444,0.004937,0.006085,0.000805,0.007374,0.010047,...,0.014272,0.007574,0.004056,0.008326,0.005630,-0.002053,0.004364,0.002990,0.006908,0.001064
2024-12-26,-0.002777,0.003176,-0.001851,0.000846,0.003425,0.007222,0.000973,0.004223,-0.004297,-0.002421,...,0.015041,-0.002265,0.000792,0.002420,0.007187,0.002860,-0.000395,0.000314,0.002066,0.002310
